In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import numpy as np
from datetime import datetime

from faster_rcnn import network
from faster_rcnn.faster_rcnn import FasterRCNN, RPN
from faster_rcnn.utils.timer import Timer

import faster_rcnn.roi_data_layer.roidb as rdl_roidb
from faster_rcnn.roi_data_layer.layer import RoIDataLayer
from faster_rcnn.datasets.factory import get_imdb
from faster_rcnn.fast_rcnn.config import cfg, cfg_from_file

try:
    from termcolor import cprint
except ImportError:
    cprint = None

try:
    from pycrayon import CrayonClient
except ImportError:
    CrayonClient = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)

In [3]:
# hyper-parameters
# ------------
#imdb_name = 'voc_2007_trainval'
imdb_name = 'kittipose_train'
cfg_file = 'experiments/cfgs/faster_rcnn_end2end.yml'
pretrained_model = '/home/pculbert/Documents/faster_rcnn_pytorch/VGG_imagenet.npy'
output_dir = 'trained_models/saved_pose_model0'

start_step = 0
end_step = 100000
lr_decay_steps = {60000, 80000}
lr_decay = 1./10

rand_seed = 1024
_DEBUG = True
use_tensorboard = False
remove_all_log = False   # remove all historical experiments in TensorBoard
exp_name = None # the previous experiment name in TensorBoard

# ------------

if rand_seed is not None:
    np.random.seed(rand_seed)

In [4]:
# load config
cfg_from_file(cfg_file)
lr = cfg.TRAIN.LEARNING_RATE
momentum = cfg.TRAIN.MOMENTUM
weight_decay = cfg.TRAIN.WEIGHT_DECAY
disp_interval = cfg.TRAIN.DISPLAY
log_interval = cfg.TRAIN.LOG_IMAGE_ITERS

In [5]:
# load data
imdb = get_imdb(imdb_name)
rdl_roidb.prepare_roidb(imdb)
roidb = imdb.roidb
# roidb is a database, containing bounding box info for all training images
data_layer = RoIDataLayer(roidb, imdb.num_classes)

<bound method kittipose.default_roidb of <faster_rcnn.datasets.kittipose.kittipose object at 0x7f907d3a5110>>
Remove empty annotations:  005066 004339 004040 003383 001752 001091 000547 Done. 
kittipose_train gt roidb loaded from /home/pculbert/Documents/faster_rcnn_pytorch/data/cache/kittipose_train_gt_roidb.pkl


In [6]:
# load net
net = FasterRCNN(classes=imdb.classes, debug=_DEBUG)
network.weights_normal_init(net, dev=0.01)
network.load_pretrained_npy(net, pretrained_model)
# model_file = '/media/longc/Data/models/VGGnet_fast_rcnn_iter_70000.h5'
# model_file = 'models/saved_model3/faster_rcnn_60000.h5'
# network.load_net(model_file, net)
# exp_name = 'vgg16_02-19_13-24'
# start_step = 60001
lr /= 10.
# network.weights_normal_init([net.bbox_fc, net.score_fc, net.fc6, net.fc7], dev=0.01)

net.cuda()
net.train()

params = list(net.parameters())
# optimizer = torch.optim.Adam(params[-8:], lr=lr)
optimizer = torch.optim.SGD(params[8:], lr=lr, momentum=momentum, weight_decay=weight_decay)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# tensorboad
use_tensorboard = use_tensorboard and CrayonClient is not None
if use_tensorboard:
    cc = CrayonClient(hostname='127.0.0.1')
    if remove_all_log:
        cc.remove_all_experiments()
    if exp_name is None:
        exp_name = datetime.now().strftime('vgg16_%m-%d_%H-%M')
        exp = cc.create_experiment(exp_name)
    else:
        exp = cc.open_experiment(exp_name)
        
#for param in net.parameters():
#    print(type(param.data), param.size())

In [7]:
# training

file_rpn_ce = open(output_dir+'/loss_rpn_ce.txt', 'w')
file_rpn_box = open(output_dir+'/loss_rpn_box.txt', 'w')
file_rcnn_ce = open(output_dir+'/loss_rcnn_ce.txt', 'w')
file_rcnn_box = open(output_dir+'/loss_rcnn_box.txt', 'w')

train_loss = 0.
tp, tf, fg, bg = 0., 0., 0., 0.
step_cnt = 0.
re_cnt = False
t = Timer()
t.tic()
#for step in range(start_step, end_step+1):
for step in range(200):

    # get one batch
    blobs = data_layer.forward()
    im_data = blobs['data'] # one image, shape = (1, H, W, 3)
    im_info = blobs['im_info'] # stores H, W, scale
    gt_boxes = blobs['gt_boxes'] # groundtruth boxes
    gt_poses = blobs['gt_poses']
    gt_ishard = blobs['gt_ishard']
    dontcare_areas = blobs['dontcare_areas']
    dontcare_poses = blobs['dontcare_poses']
    disp_data = blobs['data_disp'] # disparity map

    # forward
    net(im_data, im_info, disp_data, gt_boxes, gt_poses, gt_ishard, dontcare_areas, dontcare_poses)
    loss = net.loss + net.rpn.loss

    if _DEBUG:
        tp += float(net.tp)
        tf += float(net.tf)
        fg += net.fg_cnt
        bg += net.bg_cnt

    train_loss += loss.data[0]
    step_cnt += 1

    # backward
    optimizer.zero_grad()
    loss.backward()
    network.clip_gradient(net, 10.)
    optimizer.step()
    
    # log losses to file
    file_rpn_ce.write(str(net.rpn.cross_entropy.data.cpu().numpy()))
    file_rpn_ce.write('\n')
    file_rpn_box.write(str(net.rpn.loss_box.data.cpu().numpy()))
    file_rpn_box.write('\n')
    file_rcnn_ce.write(str(net.cross_entropy.data.cpu().numpy()))
    file_rcnn_ce.write('\n')
    file_rcnn_box.write(str(net.loss_box.data.cpu().numpy()))
    file_rcnn_box.write('\n')

    #if step % disp_interval == 0:
    if step % 1 == 0:
        duration = t.toc(average=False)
        fps = step_cnt / duration

        log_text = 'step %d, image: %s, loss: %.4f, fps: %.2f (%.2fs per batch)' % (
            step, blobs['im_name'], train_loss / step_cnt, fps, 1./fps)
        log_print(log_text, color='green', attrs=['bold'])

        if _DEBUG:
            #log_print('\tTP: %.2f%%, TF: %.2f%%, fg/bg=(%d/%d)' % (tp/float(fg*100.), tf/float(bg*100.), fg/float(step_cnt), bg/float(step_cnt)))
            log_print('\trpn_cls: %.4f, rpn_box: %.4f, rcnn_cls: %.4f, rcnn_box: %.4f, rcnn_pose: %.4f' % (
                net.rpn.cross_entropy.data.cpu().numpy(), net.rpn.loss_box.data.cpu().numpy(),
                net.cross_entropy.data.cpu().numpy(), net.loss_box.data.cpu().numpy(), net.loss_pose.data.cpu().numpy())
            )
        re_cnt = True

    if use_tensorboard and step % log_interval == 0:
        exp.add_scalar_value('train_loss', train_loss / step_cnt, step=step)
        exp.add_scalar_value('learning_rate', lr, step=step)
        if _DEBUG:
            exp.add_scalar_value('true_positive', tp/float(fg*100.), step=step)
            exp.add_scalar_value('true_negative', tf/float(bg*100.), step=step)
            losses = {'rpn_cls': float(net.rpn.cross_entropy.data.cpu().numpy()),
                      'rpn_box': float(net.rpn.loss_box.data.cpu().numpy()),
                      'rcnn_cls': float(net.cross_entropy.data.cpu().numpy()),
                      'rcnn_box': float(net.loss_box.data.cpu().numpy()),
                      'rcnn_pose': float(net.loss_pose.data.cpu().numpy())}
            exp.add_scalar_dict(losses, step=step)

    if (step % 10000 == 0) and step > 0:
        save_name = os.path.join(output_dir, 'faster_rcnn_{}.h5'.format(step))
        network.save_net(save_name, net)
        print('save model: {}'.format(save_name))
    if step in lr_decay_steps:
        lr *= lr_decay
        optimizer = torch.optim.SGD(params[8:], lr=lr, momentum=momentum, weight_decay=weight_decay)

    if re_cnt:
        tp, tf, fg, bg = 0., 0., 0, 0
        train_loss = 0
        step_cnt = 0
        t.tic()
        re_cnt = False
        
file_rpn_ce.close()
file_rpn_box.close()
file_rcnn_ce.close()
file_rcnn_box.close()

faster_rcnn/faster_rcnn.py:76: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob = F.softmax(rpn_cls_score_reshape)
faster_rcnn/faster_rcnn.py:261: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)
/home/pculbert/.virtualenvs/temp/lib/python2.7/site-packages/ipykernel_launcher.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


step 0, image: 000356.jpg, loss: 220.8041, fps: 2.90 (0.34s per batch)
	rpn_cls: 0.5921, rpn_box: 1.7224, rcnn_cls: 6.7181, rcnn_box: 19.6240, rcnn_pose: 0.0298
step 1, image: 002457.jpg, loss: 103.4450, fps: 3.99 (0.25s per batch)
	rpn_cls: 0.7366, rpn_box: 1.9560, rcnn_cls: 4.5297, rcnn_box: 7.8044, rcnn_pose: 0.5746
step 2, image: 002830.jpg, loss: 180.7328, fps: 4.07 (0.25s per batch)
	rpn_cls: 0.6828, rpn_box: 0.7027, rcnn_cls: 9.0641, rcnn_box: 16.3518, rcnn_pose: 0.4411
step 3, image: 003728.jpg, loss: 94.6832, fps: 3.93 (0.25s per batch)
	rpn_cls: 0.6547, rpn_box: 3.1288, rcnn_cls: 5.1460, rcnn_box: 5.7563, rcnn_pose: 0.0311
step 4, image: 004072.jpg, loss: 92.3013, fps: 3.92 (0.26s per batch)
	rpn_cls: 0.6693, rpn_box: 1.8611, rcnn_cls: 4.9854, rcnn_box: 6.7288, rcnn_pose: 0.7476
step 5, image: 002123.jpg, loss: 200.9737, fps: 3.99 (0.25s per batch)
	rpn_cls: 0.6730, rpn_box: 1.5226, rcnn_cls: 6.7180, rcnn_box: 17.8212, rcnn_pose: 0.1455
step 6, image: 001734.jpg, loss: 170.95

step 52, image: 003758.jpg, loss: 28.8271, fps: 3.92 (0.26s per batch)
	rpn_cls: 0.6877, rpn_box: 0.8251, rcnn_cls: 2.1541, rcnn_box: 1.7715, rcnn_pose: 0.0194
step 53, image: 001709.jpg, loss: 41.8120, fps: 3.96 (0.25s per batch)
	rpn_cls: 0.6829, rpn_box: 0.8029, rcnn_cls: 2.3695, rcnn_box: 3.0713, rcnn_pose: 0.0178
step 54, image: 000749.jpg, loss: 30.5596, fps: 4.01 (0.25s per batch)
	rpn_cls: 0.6823, rpn_box: 1.3290, rcnn_cls: 2.7096, rcnn_box: 1.3738, rcnn_pose: 0.1392


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [8]:
import gc
for obj in gc.get_objects():
    if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
        print(type(obj), obj.size())

(<class 'torch.nn.parameter.Parameter'>, torch.Size([128, 128, 3, 3]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([128]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([512, 640, 3, 3]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([512]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([18, 512, 1, 1]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([18]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([36, 512, 1, 1]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([36]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([4096, 31360]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([4096]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([4096, 4096]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([4096]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([4, 4096]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([4]))
(<class 'torch.nn.parameter.Parameter'>, torch.Size([16, 4096]))
(<class 'torch.nn.parameter.Paramete